Les Misérables

## llm 설정

In [16]:
from langchain_community.chat_models import ChatOllama
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv,find_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatMessagePromptTemplate,ChatPromptTemplate,SystemMessagePromptTemplate,HumanMessagePromptTemplate,MessagesPlaceholder
from langchain import LLMChain
load_dotenv()

True

In [ ]:
llm_model = ChatOllama(model='llama3.1:latest')
#response_llm = llm_model.invoke("Hello, how can I help you today?")
#print(response_llm.content)


In [17]:
gpt_model = ChatOpenAI(model_name="gpt-4")
#response_gpt = gpt_model.invoke("Hello, how can I help you today?")
#print(response_gpt.content)

## 1. llm 모델 작성

### 1-1. 프롬프트 작성

In [18]:
system_message = """
    너는 책을 읽어주는 어시스턴트야. 아래 지침에 따라 답변을 해줘야 해.
    모든 출력은 한국어로 해야 해.
    
    1. 사용자가 txt 파일 내용을 제공하면, 해당 내용을 문단별로 출력해야 해.
    2. 사용자가 '다음'이라는 질문을 하면, 다음 문단을 원문대로 출력해야 해.
    3. txt파일 내용을 처음 받으면, {context_character}의 말투를 참고하여 등장인물처럼 대화해야 해.
    4. 문단 중에 " ", ' ' 로 감싸진 대화문이 나오면, 대화문 뒤에 : {emotion}을 함께 출력해야 해.
    5. 사용자가 책의 내용에 대해 질문을 하면, 등장인물의 말투로 {context_books}를 참고하여 답변해야 해.

"""

In [19]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        MessagesPlaceholder(variable_name="history"), # 이전 대화 내용 저장용
        ("human", "{input}")
    ]
)
output_parser= StrOutputParser()#글자만 가져온다


### 1-2 대화 내용을 기억하는 llm 만들기

In [20]:
# 대화 메모리 생성 (대화 히스토리를 관리)
conversation_memory = ConversationBufferMemory()


# LLMChain 생성: 프롬프트와 LLM을 연결
llm_chain = LLMChain(
    llm=gpt_model,
    prompt=chat_prompt,
)

# 사용자 질문 처리 및 답변 생성 함수
def handle_user_input(user_input, history=[], context_books="책 내용 예시", context_character="장발장", emotion="중립"):
    #global PARAGRAPH_INDEX
    
    # 히스토리 업데이트
    history.append({"role": "user", "content": user_input})
    '''
    if txt_path is not None:
        with open(txt_path, 'r', encoding='utf-8') as file:
            file_content = file.read()

        # 문단별로 나누기
        paragraphs = file_content.split('\n\n')  # 빈 줄을 기준으로 문단 분리

        if paragraph_index < len(paragraphs):
            user_input = paragraphs[paragraph_index]  # 해당 인덱스의 문단을 user_input으로 설정
            PARAGRAPH_INDEX +=1
        else:
            user_input = "모든 문단이 끝났습니다."'''
            
    # 모델 실행 및 답변 생성
    response = llm_chain.run({
        "history": history,
        "input": user_input,
        "context_books": context_books,
        "context_character": context_character,
        "emotion": emotion,
    })
    
    # 히스토리에 응답 추가
    history.append({"role": "assistant", "content": response})
    
    return response, history
'''
# 예시 실행
user_input = "첫 문단을 읽어줘"
history = []
#paragraph_index = 3  # 첫 번째 문단부터 시작
response, history = handle_user_input(user_input, history,txt_path="books/LES MISERABLES_ch1.txt")
print(response)
print,history
'''

'\n# 예시 실행\nuser_input = "첫 문단을 읽어줘"\nhistory = []\n#paragraph_index = 3  # 첫 번째 문단부터 시작\nresponse, history = handle_user_input(user_input, history,txt_path="books/LES MISERABLES_ch1.txt")\nprint(response)\nprint,history\n'

### 2. 텍스트 벡터화 임베딩

### 챗봇 ui

### 2-1 백터화 준비

In [21]:
from langchain_openai import OpenAIEmbeddings
import os
from langchain.docstore.document import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
import shutil
import io
OPENAI_DB_PATH = "./openai_db" # 벡터화 임베딩 데이터 저장 장소
UPLOAD_TXT_PATH = "" # 업로드한 파일의 경로

#감정 리스트
emotion_list = []
emb = OpenAIEmbeddings()
docs=[]
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, # 각 조각의 최대 문자 수를 1000자로 설정합니다.
    chunk_overlap = 100  # 각 조각이 서로 겹치는 부분을 100자로 설정하여 분할된 텍스트 간의 연결성을 유지합니다.
    )
openai_db = None

### 2-2 파일 업로드 처리

In [31]:
save_dir = "save_txt"
def save_uploaded_file(file):
    global LLM_HISTORY
    os.makedirs(save_dir, exist_ok=True)
    file_path = os.path.join(save_dir, os.path.basename(file.name))
    if file is not None:
        # file 은 임시 저장 결로를 나타내는 듯
        shutil.copy(file, file_path)
    print("저장되었음")
    PARAGRAPH_INDEX = 0

    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
        paragraphs = content.split('\n\n')  # 빈 줄로 문단 분리
        for paragraph in paragraphs:
            #LLM_HISTORY.append({"role": "assistant", "content": paragraph})
            LLM_HISTORY.append(paragraph)
    return file_path

In [32]:
# 파일 벡터화 임베딩
def text_vectorize(file):
    
    file_path = os.path.join(save_dir, os.path.basename(file.name))

    if not os.path.exists(file_path):
            print(f"File not found at: {file_path}")
            return None

    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            doc = Document(page_content=content)
            docs.append(doc)
            docs_spliter = text_splitter.split_documents(docs)
            openai_vectorstore = Chroma.from_documents(
                docs_spliter,
                emb,
                persist_directory=OPENAI_DB_PATH
            )
    except Exception as e:
        print(f"Error reading file: {e}")
        return None
    

    # 데이터가 잘 저장되었는지 확인하는 부분
    try:
        doc_count = openai_vectorstore._collection.count()
        print(f"벡터 저장소에 저장된 문서 수: {doc_count}")
        
        # 예시로 첫 번째 문서 임베딩을 확인해보기
        if doc_count > 0:
            sample_result = openai_vectorstore.similarity_search(docs_spliter[0].page_content)
            print(f"첫 번째 문서와 유사한 문서 검색 결과: {sample_result}")
    except Exception as e:
        print(f"Error verifying vector store: {e}")
        return None

    return openai_vectorstore

In [33]:
# 파일 업로드 처리 함수
# 이거 수정해서 벡터화 및 파일 db 생성해서 순서대로 부르게 하기
# gpt 보라
def chatbot_send_file(chat_history, file):
    global UPLOAD_TXT_PATH
    global CHATBOT

    UPLOAD_TXT_PATH = save_uploaded_file(file)
    openai_db = text_vectorize(file)
    
    chat_history.append(["사용자가 업로드한 파일입니다.",f"{file.name}"])
    return chat_history

In [41]:
# 이벤트 처리 함수
def chatbot_send_message(cb_chatbot,cb_textbox):
    global CONTEXT_BOOKS
    global LLM_HISTORY
    global PARAGRAPH_INDEX
    # 사용자 입력 가져오기
    user_input = cb_textbox
    
    if openai_db is not None:
        openai_retriever = openai_db.as_retriever(
        search_kwargs={"k":5}
        )
    
        CONTEXT_BOOKS = openai_retriever.invoke(user_input)

    # handle_user_input 함수를 통해 답변 생성
    response, updated_history = handle_user_input(user_input, history=LLM_HISTORY,context_books= CONTEXT_BOOKS)
    #response, updated_history = CHATBOT.handle_user_input(user_input, history=LLM_HISTORY,context_books= CONTEXT_BOOKS)

    #PARAGRAPH_INDEX +=1
    
    # 챗봇의 응답을 대화 기록에 추가
    LLM_HISTORY = updated_history
    most_recent_user = None
    most_recent_assistant = None
    chatbot_output = []
    for entry in reversed(LLM_HISTORY):
        if entry["role"] == "user" and most_recent_user is None:
            most_recent_user = entry["content"]
        elif entry["role"] == "assistant" and most_recent_assistant is None:
            most_recent_assistant = entry["content"]
        
        # 둘 다 찾으면 종료
        if most_recent_user and most_recent_assistant:
            break
            
    chatbot_output.append((most_recent_user, most_recent_assistant))

    # 텍스트박스 초기화
    cb_textbox = ""
    print(LLM_HISTORY)
    print(chatbot_output)
    # 새로운 대화 기록과 초기화된 텍스트박스를 반환
    return chatbot_output, cb_textbox

In [42]:
import gradio as gr

# 프롬프트에 들어가야 하는 내용
LLM_HISTORY = []
CONTEXT_BOOKS = '예시' # 임베딩된 내용 불러오게 하기
CONTEXT_CHARACTER = '' # 임베딩된 캐릭터 말하는 방식 불러오게 하기
QUESTION = ''
EMOTION = '' # 대화문 추출 후 감정 분석하기 
PARAGRAPH_INDEX = 0
TXT = ''
CHATBOT = None

with gr.Blocks() as app:
    with gr.Tab("음성 인식봇"):
        with gr.Column():
            gr.Markdown(
                value="""
<center><h1> 음성 인식 봇 </h1></center>
<center> AI 인공지능 봇입니다. 음성으로 묻거나 문서 요약, 일정 관리 등을 할 수 있습니다.</center>
"""
            )
            cb_chatbot = gr.Chatbot(
                value=[[None, "안녕하세요 AI 챗봇입니다. 무엇이든 질문하세요"],
                       ['나는 지금 배가 고파요', '배가 고프시군요 메뉴를 추천해드릴까요?']
                       ],
                label="구미챗봇",
            )
        with gr.Row():
            cb_textbox = gr.Textbox(
                lines=3,
                placeholder="입력창입니다.",
                container=False,
                scale=7
            ) # 채팅 입력

            gr.Audio(
                format="mp3",
                scale=1,
                min_width=200,
                label="음성을 입력해주세요"
            ) # 음성 입력

            cb_send_btn = gr.Button(
                value="보내기",
                visible="primary",
                scale=1
            ) # 실행 버튼
            
            file_upload = gr.File(
                label="txt 파일을 업로드해주세요",
                type="filepath"
            ) # 파일 업로드

        with gr.Row():
            gr.Button(
                value="되돌리기"
            )
            gr.Button(
                value="초기화"
            )
        
        # 텍스트 입력 처리
        cb_send_btn.click(
            fn=chatbot_send_message,
            inputs=[cb_chatbot, cb_textbox],
            outputs=[cb_chatbot, cb_textbox]
        )
        
        # 파일 업로드 처리
        file_upload.change(
            fn= chatbot_send_file,
            inputs = [cb_chatbot,file_upload],#입력값 전달될 것 
            outputs=cb_chatbot#출력값 들어갈 곳
        )

    with gr.Tab("문서 요약봇"):
        pass
    with gr.Tab("일정 관리봇"):
        pass

app.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


저장되었음
벡터 저장소에 저장된 문서 수: 17476
첫 번째 문서와 유사한 문서 검색 결과: [Document(page_content='CHAPTER I—M. MYRIEL\nIn 1815, M. Charles-François-Bienvenu Myriel was Bishop of D—— He was an old man of about seventy-five years of age; he had occupied the see of D—— since 1806.'), Document(page_content='CHAPTER I—M. MYRIEL\nIn 1815, M. Charles-François-Bienvenu Myriel was Bishop of D—— He was an old man of about seventy-five years of age; he had occupied the see of D—— since 1806.'), Document(page_content='CHAPTER I—M. MYRIEL\nIn 1815, M. Charles-François-Bienvenu Myriel was Bishop of D—— He was an old man of about seventy-five years of age; he had occupied the see of D—— since 1806.'), Document(page_content='CHAPTER I—M. MYRIEL\nIn 1815, M. Charles-François-Bienvenu Myriel was Bishop of D—— He was an old man of about seventy-five years of age; he had occupied the see of D—— since 1806.')]
['LES MISÉRABLES\nPREFACE\nSo long as there shall exist, by virtue of law and custom, decrees of damnation pronounced 

In [ ]:
import gradio as gr
import shutil

def save_file(file):
    if file is not None:
        # file 은 임시 저장 결로를 나타내는 듯
        shutil.copy(file, "save_file.txt")

with gr.Blocks() as app:
    file_input = gr.File(label="업로드 파일")
    send_btn = gr.Button("파일 저장하기")

    send_btn.click(fn=save_file, inputs=file_input, outputs=None)

app.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
